In [1]:
!pip install kaggle

In [2]:
from typing import Iterable, Callable, Dict, Tuple
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience
from numpy.typing import ArrayLike
import data_utils as du

In [3]:
!~/.local/bin/kaggle datasets download -d rohitrox/healthcare-provider-fraud-detection-analysis -p data

healthcare-provider-fraud-detection-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!cd data && unzip -n '*.zip'

Archive:  healthcare-provider-fraud-detection-analysis.zip


In [5]:
DATA_DIR = 'data/'

In [6]:
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os

load_dotenv('config.env', override=True)
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

gds.set_database("neo4j")

In [7]:
provider_feat_df = pd.read_csv(DATA_DIR + 'baseline.csv')
provider_feat_df['providerId'] = provider_feat_df.provider.str.slice(start=3).astype(int)
provider_feat_df.to_csv(DATA_DIR + 'baseline.csv', index=False)
provider_feat_df

,provider,potentialFraudInd,inpationFraction,renalDiseaseIndicatorNumEnc,chronicCondAlzheimerEnc,chronicCondHeartfailureEnc,chronicCondKidneyDiseaseEnc,chronicCondCancerEnc,chronicCondObstrPulmonaryEnc,chronicCondDepressionEnc,...,chronicCondObstrPulmonaryIndEnc,chronicCondDepressionIndEnc,chronicCondDiabetesIndEnc,chronicCondIschemicHeartIndEnc,chronicCondOsteoporasisIndEnc,chronicCondrheumatoidarthritisIndEnc,chronicCondstrokeIndEnc,claimCount,avgClaimAmtReimbursed,providerId
0,PRV51001,0,0.200000,0.027972,0.052448,0.066434,0.059441,0.017483,0.034965,0.031469,...,0.034965,0.031469,0.073427,0.080420,0.020979,0.027972,0.020979,25,4185.600000,51001
1,PRV51003,1,0.469697,0.023596,0.045566,0.065094,0.052075,0.008137,0.033360,0.043938,...,0.033360,0.043938,0.081367,0.091131,0.026851,0.030919,0.009764,132,4588.409091,51003
2,PRV51004,0,0.000000,0.017464,0.048595,0.066819,0.037965,0.012149,0.031131,0.047836,...,0.031131,0.047836,0.079727,0.082005,0.037206,0.034928,0.012908,149,350.134228,51004
3,PRV51005,1,0.000000,0.024998,0.041116,0.065631,0.048934,0.015925,0.028472,0.046810,...,0.028472,0.046810,0.077116,0.086382,0.033201,0.031947,0.011968,1165,241.124464,51005
4,PRV51007,0,0.041667,0.018003,0.042553,0.065466,0.036007,0.019640,0.026187,0.047463,...,0.026187,0.047463,0.080196,0.083470,0.034370,0.036007,0.019640,72,468.194444,51007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,0,0.000000,0.016722,0.046823,0.066890,0.050167,0.013378,0.036789,0.030100,...,0.036789,0.030100,0.066890,0.093645,0.043478,0.030100,0.013378,28,380.000000,57759
5406,PRV57760,0,0.000000,0.000000,0.019481,0.071429,0.012987,0.000000,0.012987,0.045455,...,0.012987,0.045455,0.116883,0.142857,0.071429,0.006494,0.000000,22,216.818182,57760
5407,PRV57761,0,0.000000,0.028152,0.044064,0.068543,0.048960,0.017136,0.036720,0.046512,...,0.036720,0.046512,0.067319,0.075887,0.039168,0.029376,0.012240,82,225.243902,57761
5408,PRV57762,0,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.100000,0.000000,...,0.100000,0.000000,0.100000,0.100000,0.000000,0.000000,0.000000,1,1900.000000,57762


In [19]:
claim_df = du.make_in_out_df(DATA_DIR)
claim_df['providerId'] = claim_df.provider.str.slice(start=3).astype(int)
claim_df

,beneID,claimID,claimStartDt,claimEndDt,provider,inscClaimAmtReimbursed,attendingPhysician,operatingPhysician,otherPhysician,admissionDt,...,clmDiagnosisCode9,clmDiagnosisCode10,clmProcedureCode1,clmProcedureCode2,clmProcedureCode3,clmProcedureCode4,clmProcedureCode5,clmProcedureCode6,inpatient,providerId
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,55912
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,1,55907
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,56046
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,1,52405
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,1,56614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517732,BENE159198,CLM510792,2009-08-06,2009-08-06,PRV53699,800,PHY364188,PHY364188,PHY385752,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,53699
517733,BENE159198,CLM551294,2009-08-29,2009-08-29,PRV53702,400,PHY423019,PHY332284,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,53702
517734,BENE159198,CLM596444,2009-09-24,2009-09-24,PRV53676,60,PHY361063,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,53676
517735,BENE159198,CLM636992,2009-10-18,2009-10-18,PRV53689,70,PHY403198,NaN,PHY419379,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,53689


## Nodes

In [10]:
gds.run_cypher('CREATE CONSTRAINT unique_provider_id IF NOT EXISTS FOR (n:Provider) REQUIRE n.providerId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_beneficiary_id IF NOT EXISTS FOR (n:Beneficiary) REQUIRE n.beneId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_diagnosis_code IF NOT EXISTS FOR (n:DiagnosisCode) REQUIRE n.code IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_diagnosis_group_code IF NOT EXISTS FOR (n:DiagnosisGroupCode) REQUIRE n.code IS UNIQUE')

""


### Load Nodes

In [11]:
print('======  loading provider nodes  ======')
node_records = provider_feat_df[['providerId', 'potentialFraudInd']].to_dict('records')

res = gds.run_cypher('''
    UNWIND $data AS d
    MERGE(n:Provider {providerId: d.providerId})
    SET n.potentialFraudInd = d.potentialFraudInd
    RETURN count(n) AS nodeLoadedCount
''', params={'data': node_records})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes.')


======  loading provider nodes  ======
Complete: Loaded 5,410 nodes.


In [12]:
### Beneficiary Nodes
print('======  loading benificiary nodes  ======')
res = gds.run_cypher('''
    UNWIND $nodeKeys AS nodeKey
    MERGE(n:Beneficiary {beneId: nodeKey})
    RETURN count(n) AS nodeLoadedCount
''', params={'nodeKeys': claim_df.beneID.drop_duplicates().tolist()})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes.')

======  loading benificiary nodes  ======
Complete: Loaded 138,556 nodes.


In [13]:
### Diagnosis Claim Code Nodes
diagnosis_code_list = []
for col in claim_df.columns:
    if 'diagnosiscode' in col.lower():
        diagnosis_code_list.extend(claim_df[col].dropna().drop_duplicates().tolist())

print('======  loading dianosis code nodes  ======')
res = gds.run_cypher('''
    UNWIND $nodeKeys AS nodeKey
    MERGE(n:DiagnosisCode {code: nodeKey})
    RETURN count(n) AS nodeLoadedCount
''', params={'nodeKeys': pd.Series(diagnosis_code_list).drop_duplicates().tolist()})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes') 

======  loading dianosis code nodes  ======
Complete: Loaded 11,040 nodes


In [14]:
### Diagnosis Group Code
print('======  loading dianosis group code nodes  ======')
res = gds.run_cypher('''
    UNWIND $nodeKeys AS nodeKey
    MERGE(n:DiagnosisGroupCode {code: nodeKey})
    RETURN count(n) AS nodeLoadedCount
''', params={'nodeKeys': claim_df['diagnosisGroupCode'].dropna().drop_duplicates().tolist()})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes')

======  loading dianosis group code nodes  ======
Complete: Loaded 736 nodes


In [15]:
## Relationships

In [20]:
### Beneficiary Relationships
print(f'======  loading HAS_CLAIMS_WITH_BENE relationships  ======')
tmp_df = claim_df[['providerId', 'beneID', 'claimID', 'inscClaimAmtReimbursed', 'inpatient']] \
    .groupby(['providerId', 'beneID']).agg({
        'claimID': 'count',
        'inscClaimAmtReimbursed': sum,
        'inpatient': sum
    }).reset_index()
rel_records = tmp_df.to_dict('records')
total = len(rel_records)
print(f'staged {total:,} records...')

cumulative_count = 0
for d in du.chunks(rel_records):
    res = gds.run_cypher('''
        UNWIND $data AS d
        MATCH(p:Provider {providerId: d.providerId})
        MATCH(t:Beneficiary {beneId: d.beneID})
        MERGE (p)-[r:HAS_CLAIMS_WITH_BENE]->(t)
        SET r.claimCount = d.claimID,
            r.inscClaimReimbursedSum = d.inscClaimAmtReimbursed,
            r.inpatientCount = d.inpatient
        RETURN count(r) AS relLoadedCount
    ''', params={'data': d})
    cumulative_count += res.relLoadedCount[0]
    print(f'Loaded {cumulative_count:,} of {total:,} relationships...')
print('Complete')

======  loading HAS_CLAIMS_WITH_BENE relationships  ======
staged 363,300 records...
Loaded 50,000 of 363,300 relationships...
Loaded 100,000 of 363,300 relationships...
Loaded 150,000 of 363,300 relationships...
Loaded 200,000 of 363,300 relationships...
Loaded 250,000 of 363,300 relationships...
Loaded 300,000 of 363,300 relationships...
Loaded 350,000 of 363,300 relationships...
Loaded 363,300 of 363,300 relationships...
Complete


In [24]:
def load_claim_rels(gds: GraphDataScience, rel_type: str, target_node_label: str, clm_df: pd.DataFrame,
                    target_key_map: Tuple):
    print(f'======  loading {rel_type} relationships  ======')
    tmp_df = clm_df[['providerId', target_key_map[0], 'claimID', 'inscClaimAmtReimbursed', 'inpatient']] \
        .groupby(['providerId', target_key_map[0]]).agg({
            'claimID': 'count',
            'inscClaimAmtReimbursed': sum,
            'inpatient': sum
        }).reset_index()
    rel_records = tmp_df.to_dict('records')
    total = len(rel_records)
    print(f'staging {total:,} records')

    cumulative_count = 0
    for d in du.chunks(rel_records):
        res = gds.run_cypher(f'''
            UNWIND $data AS d
            MATCH(p:Provider {{providerId: d.providerId}})
            MATCH(t:{target_node_label} {{{target_key_map[1]}: d.{target_key_map[0]}}})
            MERGE (p)-[r:{rel_type}]->(t)
            SET r.claimCount = d.claimID,
                r.inscClaimReimbursedSum = d.inscClaimAmtReimbursed,
                r.inpatientCount = d.inpatient
            RETURN count(r) AS relLoadedCount
        ''', params={'data': d})
        cumulative_count += res.iloc[0, 0]
        print(f'Loaded {cumulative_count:,} of {total:,} relationships')

In [25]:
### Group Diagnosis Code Relationships
load_claim_rels(gds, 'HAS_CLAIMS_WITH_GROUP_CODE', 'DiagnosisGroupCode', claim_df, ('diagnosisGroupCode', 'code'))

======  loading HAS_CLAIMS_WITH_GROUP_CODE relationships  ======
staging 37,553 records
Loaded 37,553 of 37,553 relationships


In [27]:
### Diagnosis Code Relationships
diagnosis_clm_code_dfs = []
for col in claim_df.columns:
    if 'diagnosiscode' in col.lower():
        tmp_clm_df = claim_df[['providerId', 'claimID', 'inscClaimAmtReimbursed', 'inpatient', col]]\
            .dropna(subset=[col]).rename(columns={col: 'code'})
        diagnosis_clm_code_dfs.append(tmp_clm_df)
diagnosis_clm_code_df = pd.concat(diagnosis_clm_code_dfs)
load_claim_rels(gds, 'HAS_CLAIMS_WITH_CODE', 'DiagnosisCode', diagnosis_clm_code_df, ('code', 'code'))

======  loading HAS_CLAIMS_WITH_CODE relationships  ======
staging 1,001,352 records
Loaded 50,000 of 1,001,352 relationships
Loaded 100,000 of 1,001,352 relationships
Loaded 150,000 of 1,001,352 relationships
Loaded 200,000 of 1,001,352 relationships
Loaded 250,000 of 1,001,352 relationships
Loaded 300,000 of 1,001,352 relationships
Loaded 350,000 of 1,001,352 relationships
Loaded 400,000 of 1,001,352 relationships
Loaded 450,000 of 1,001,352 relationships
Loaded 500,000 of 1,001,352 relationships
Loaded 550,000 of 1,001,352 relationships
Loaded 600,000 of 1,001,352 relationships
Loaded 650,000 of 1,001,352 relationships
Loaded 700,000 of 1,001,352 relationships
Loaded 750,000 of 1,001,352 relationships
Loaded 800,000 of 1,001,352 relationships
Loaded 850,000 of 1,001,352 relationships
Loaded 900,000 of 1,001,352 relationships
Loaded 950,000 of 1,001,352 relationships
Loaded 1,000,000 of 1,001,352 relationships
Loaded 1,001,352 of 1,001,352 relationships
